In [1]:
import pandas as pd

In [56]:
claims = pd.read_csv('data/k12_with_budget_and_resource.csv')

dem = pd.read_csv('data/k-12_variables_county.csv')

cc = pd.read_csv('data/city_county_mapping.csv')

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,25,26,29,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    dupes = all_dupes['index'].to_list()
   

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    
    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    return(df.drop(index=badi).reset_index())

In [94]:
column_list = ['TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm','Budget']
def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
    # creating a total claims column
    
    df['total_claims'] = [1]*len(df)
    
    #aggregate!
    relevant_columns = column_list + ['total_claims']
    agg = df.groupby(by).sum()[relevant_columns]
    agg['total_programs'] = tp
    
    # Resource/OBF budget as percentage of total budget
    obf_budget = df.groupby([by, 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
    resource_budget = df.groupby([by, 'Resource_Flag']).Budget.sum().unstack().fillna(0).loc[:, 1]
    agg['perc_obf_budget'] = (obf_budget/agg['Budget'])*100
    agg['perc_resource_budget'] = (resource_budget/agg['Budget'])*100
    
    return(agg.reset_index())

In [57]:
claims = claims.drop_duplicates()

claims = no_dupes(claims)

In [20]:
cc.columns

Index(['city', 'county'], dtype='object')

In [21]:
claims.columns

Index(['index', 'PrgYear', 'ClaimID', 'PrgID', 'ProgramName', 'PA', 'Sector',
       'SiteCity', 'SiteZipCode', 'SiteID', 'NAICSCode', 'BldgHVAC', 'BldgLoc',
       'BldgType', 'BldgVint', 'E3ClimateZone', 'E3GasSavProfile',
       'E3GasSector', 'E3MeaElecEndUseShape', 'E3TargetSector',
       'ImplementationID', 'InstallationDate', 'NAICSBldgType', 'NumUnits',
       'OBF_Flag', 'PrgElement', 'RateScheduleElec', 'RateScheduleGas',
       'REN_Flag', 'Residential_Flag', 'SchoolIdentifier',
       'TotalFirstYearGrosskW', 'TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm', 'TotalGrossIncentive',
       'TotalGrossMeasureCost', 'TotalGrossMeasureCost_ER',
       'TotalLifecycleGrosskW', 'TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm', 'Upstream_Flag', 'WaterOnly_Flag'],
      dtype='object')

In [58]:
claims = claims.merge(cc, left_on='SiteCity', right_on='city', how='left')

In [23]:
claims.columns

Index(['index', 'PrgYear', 'ClaimID', 'PrgID', 'ProgramName', 'PA', 'Sector',
       'SiteCity', 'SiteZipCode', 'SiteID', 'NAICSCode', 'BldgHVAC', 'BldgLoc',
       'BldgType', 'BldgVint', 'E3ClimateZone', 'E3GasSavProfile',
       'E3GasSector', 'E3MeaElecEndUseShape', 'E3TargetSector',
       'ImplementationID', 'InstallationDate', 'NAICSBldgType', 'NumUnits',
       'OBF_Flag', 'PrgElement', 'RateScheduleElec', 'RateScheduleGas',
       'REN_Flag', 'Residential_Flag', 'SchoolIdentifier',
       'TotalFirstYearGrosskW', 'TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm', 'TotalGrossIncentive',
       'TotalGrossMeasureCost', 'TotalGrossMeasureCost_ER',
       'TotalLifecycleGrosskW', 'TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm', 'Upstream_Flag', 'WaterOnly_Flag', 'city',
       'county'],
      dtype='object')

In [59]:
claims[claims.SiteCity != claims.city].SiteCity.value_counts()

ETIWANDA        11
ALTA LOMA        5
EASTVALE         5
LOS MOLINOS      4
SAUGUS           3
HILLSBOROUGH     2
PETROLIA         1
KENTFIELD        1
Name: SiteCity, dtype: int64

In [28]:
claims[claims.SiteCity == 'PETROLIA']

,index,PrgYear,ClaimID,PrgID,ProgramName,PA,Sector,SiteCity,SiteZipCode,SiteID,...,TotalGrossIncentive,TotalGrossMeasureCost,TotalGrossMeasureCost_ER,TotalLifecycleGrosskW,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Upstream_Flag,WaterOnly_Flag,city,county
1168,1604,2018,PGE-2018-Q2-46093,PGE211016,Redwood Coast,PGE,Com,PETROLIA,95558,PGE-1217156,...,264.220003,0.0,0.0,0.0,0.0,0.0,False,False,NaN,NaN


the above places are tiny cities that have no associated county, have to manually look them up

In [64]:
df = claims.copy()

In [69]:
tupes = [('ETIWANDA', 'SAN BERNARDINO'),('ALTA LOMA', 'SAN BERNARDINO'),('EASTVALE', 'RIVERSIDE'),('LOS MOLINOS', 'TEHAMA'),('SAUGUS','LOS ANGELES'),('HILLSBOROUGH', 'SAN MATEO'), ('PETROLIA', 'HUMBOLDT'), ('KENTFIELD', 'MARIN')]

#create index of places to add county value
for t in tupes:
    for e in df.SiteCity.items(): 
        if e[1] == t[0]:
            i = df[df.SiteCity==t[0]].index.to_list()
    print(i,t[0],t[1])
#add county to dataset
    df.loc[i, 'county'] = t[1]
    print(t)
    
print('success?')

[1237, 1248, 1263, 1296, 6494, 6518, 6537, 6538, 6539, 6540, 6541] ETIWANDA SAN BERNARDINO
('ETIWANDA', 'SAN BERNARDINO')
[2079, 4399, 6681, 6687, 6694] ALTA LOMA SAN BERNARDINO
('ALTA LOMA', 'SAN BERNARDINO')
[5536, 6167, 6927, 6929, 6933] EASTVALE RIVERSIDE
('EASTVALE', 'RIVERSIDE')
[242, 347, 371, 385] LOS MOLINOS TEHAMA
('LOS MOLINOS', 'TEHAMA')
[4093, 6680, 6698] SAUGUS LOS ANGELES
('SAUGUS', 'LOS ANGELES')
[85, 120] HILLSBOROUGH SAN MATEO
('HILLSBOROUGH', 'SAN MATEO')
[1168] PETROLIA HUMBOLDT
('PETROLIA', 'HUMBOLDT')
[79] KENTFIELD MARIN
('KENTFIELD', 'MARIN')
success?


In [70]:
df[df.county.isna()]

,index,PrgYear,ClaimID,PrgID,ProgramName,PA,Sector,SiteCity,SiteZipCode,SiteID,...,TotalGrossMeasureCost_ER,TotalLifecycleGrosskW,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Upstream_Flag,WaterOnly_Flag,Budget,Resource_Flag,city,county


whooo

aggregation time

In [95]:
k12ag = aggregate_me(df, 'county', False, column_list)

demographics time

In [79]:
dem.county_name = dem.county_name.str.upper()

In [81]:
dem.head()

,county_name,enrollment,free_meal_count,frpm_count,perc_eligible_free,perc_eligible_frpm,total_lcff_entitlement,no_title1_status,number_of_schools,perc_title1_status
0,ALAMEDA,227174,83426,95299,0.367234,0.419498,229206308,55,396,0.138889
1,ALPINE,70,41,43,0.585714,0.614286,0,0,3,0.000000
2,AMADOR,4166,1522,1664,0.365338,0.399424,0,0,15,0.000000
3,BUTTE,30037,15804,17698,0.526151,0.589207,36579973,15,87,0.172414
4,CALAVERAS,5315,2427,2641,0.456632,0.496896,3627998,0,27,0.000000


In [96]:
k12 = dem.merge(k12ag, left_on='county_name', right_on='county', how='left')

In [97]:
k12.head()

,county_name,enrollment,free_meal_count,frpm_count,perc_eligible_free,perc_eligible_frpm,total_lcff_entitlement,no_title1_status,number_of_schools,perc_title1_status,county,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,total_claims,total_programs,perc_obf_budget,perc_resource_budget
0,ALAMEDA,227174,83426,95299,0.367234,0.419498,229206308,55,396,0.138889,ALAMEDA,141267.60080,-93.93267,1.712310e+06,-1259.075064,75584.282608,38.0,4.0,14.588481,100.0
1,ALPINE,70,41,43,0.585714,0.614286,0,0,3,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMADOR,4166,1522,1664,0.365338,0.399424,0,0,15,0.000000,AMADOR,25881.36288,-48.04083,3.010994e+05,-521.086392,19733.389311,4.0,1.0,0.000000,100.0
3,BUTTE,30037,15804,17698,0.526151,0.589207,36579973,15,87,0.172414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CALAVERAS,5315,2427,2641,0.456632,0.496896,3627998,0,27,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
k12.drop(columns='county', inplace=True)

In [101]:
k12.fillna(0, inplace=True)

In [102]:
k12.to_csv('data/k12_counties_1.csv', index=False)

In [105]:
len(k12[k12.Budget==0])

26